In [1]:
import pandas as pd
import dotenv

dotenv.load_dotenv()

True

In [2]:
df_london_smartmeter = pd.read_csv(
    "../data/LondonSmartMeter/london_smart_meters_dataset_without_missing_values_first_30_consumers.csv",
    index_col="Time",
    parse_dates=["Time"],
)
df_kddcup = pd.read_csv(
    "../data/KDDCup_2018/kdd_cup_2018_dataset_without_missing_values.csv", index_col="Time", parse_dates=["Time"]
)
df_electricity_321_hourly = pd.read_csv(
    "../data/Electricity321Hourly/electricity_hourly_dataset.csv", index_col="Time", parse_dates=["Time"]
)
df_electricity_370 = pd.read_csv(
    "../data/Electricity370/LD2011_2014_first_40_consumers.csv", index_col="Time", parse_dates=["Time"]
)

In [ ]:
import torch
import psutil

# Check if CUDA is available
if torch.cuda.is_available():
    # Get the current device
    torch.set_float32_matmul_precision("high")
    device = torch.cuda.current_device()
    print(f"Using CUDA device: {torch.cuda.get_device_name(device)}")
else:
    device = "cpu"
    print("CUDA is not available")

print("Physical cores:", psutil.cpu_count(logical=False))
print("Total cores:", psutil.cpu_count(logical=True))
cpu_freq = psutil.cpu_freq()
print(f"Current Frequency: {cpu_freq.current:.2f}Mhz")

# RAM Information
svmem = psutil.virtual_memory()
print(f"Total: {svmem.total / (1024**3):.2f} GB")
print(f"Available: {svmem.available / (1024**3):.2f} GB")
print(f"Used: {svmem.used / (1024**3):.2f} GB")
print(f"Percentage: {svmem.percent}%")

print("Distributed PyTorch available:", torch.distributed.is_available())

Using CUDA device: NVIDIA L40S
Physical cores: 64
Total cores: 128
Current Frequency: 3721.55Mhz
Total: 1007.57 GB
Available: 441.48 GB
Used: 557.55 GB
Percentage: 56.2%
Distributed PyTorch available: True


In [4]:
from copy import deepcopy
from ts_inverse.models import FCN_Predictor, CNN_Predictor, GRU_Predictor, TCN_Predictor
from ts_inverse.models.jit_gru import JitSeq2Seq_Predictor
from ts_inverse.utils import grid_search_params
from ts_inverse.workers import AttackLearningToInvertWorker


def start_multi_process(g_config, a_config, d_config, m_config, def_config, pool_size):
    search_args = []
    search_configs = list(grid_search_params(g_config))
    search_attack_configs = list(grid_search_params(a_config))
    search_dataset_settings = list(grid_search_params(d_config))
    search_model_settings = list(grid_search_params(m_config))
    search_defense_settings = list(grid_search_params(def_config))
    for def_config in search_defense_settings:
        for original_g_config in search_configs:
            for a_config in search_attack_configs:
                g_config = deepcopy(original_g_config)
                g_config.update(a_config)
                for m_config in search_model_settings:
                    for d_config in search_dataset_settings:
                        fa_models_config = {
                            "features": [[0]],
                            "input_size": d_config["observation_days"],
                            "output_size": d_config["future_days"],
                        }
                        search_for_all_models_settings = list(grid_search_params(fa_models_config))
                        for fa_models_config in search_for_all_models_settings:
                            g_config["run_number"] = len(search_args)
                            args = (g_config, d_config, m_config, fa_models_config, def_config)
                            search_args.append(deepcopy(args))

    print(f"Starting {len(search_args)} processes")
    if pool_size == 1:
        for args in search_args:
            AttackLearningToInvertWorker(args[0]["run_number"]).worker_process(*args)


global_config = {
    "logger_service": "wandb",
    "experiment_name": "baseline_with_defenses_5-3-2025a",
    "seed": [10, 43, 28],
    "number_of_batches": 5000,  # Training steps
    "warmup_number_of_batches": 0,
    "attack_number_of_batches": 1,
    "batch_size": 1,
    "device": 1,
    "verbose": False,
    "pool_size": 1,
    "run_number": -1,
    "total_variation_alpha": 0,
    "after_effect": "none",
    "update_model": True,  # Update the model in generating gradients from training data
    "load_lti_model": False,
    "evaluate_trained_model": True,
}

defense_config = [
    {
        "sign": True,
        "defense_name": "sign",
    },
    {
        "prune_rate": [0.1],
        "defense_name": "prune",
    },
    {
        "dp_epsilon": [0.1],
        "defense_name": "gauss",
    },
    {
        "defense_name": "none",
    },  # No defense
]

attack_config = [
    {
        "attack_method": "LTI",
        "num_learn_epochs": 250,
        "learn_learning_rate": 1e-3,  # in terms of reconstruction SMAPE little difference, but in t_loss and v_loss, 1e-3 converges faster
        "inversion_batch_size": global_config["batch_size"],
        "attack_batch_size": 128,
        "attack_hidden_size": 3000,
        "attack_loss": "mse",
        "learn_optimizer": "adam",
        "learn_lr_decay": "75%",
        "inversion_model": "GradToInputNN",
        "attack_targets": True,
    },
    # {
    #     "attack_method": "LTI_OOD",
    #     "num_learn_epochs": 250,
    #     "learn_learning_rate": 1e-3,  # in terms of reconstruction SMAPE little difference, but in t_loss and v_loss, 1e-3 converges faster
    #     "inversion_batch_size": global_config["batch_size"],
    #     "attack_batch_size": 128,
    #     "attack_hidden_size": 3000,
    #     "attack_loss": "mse",
    #     "learn_optimizer": "adam",
    #     "learn_lr_decay": "75%",
    #     "inversion_model": "GradToInputNN",
    #     "attack_targets": True,
    #     "aux_dataset": {
    #         "dataset": "london_smartmeter",
    #         "columns": df_london_smartmeter.columns.tolist()[10:],
    #         "train_stride": 1,
    #         "observation_days": 1,
    #         "future_days": 1,
    #         "normalize": "minmax",
    #     },
    # },
]

dataset_config = [
    {
        "dataset": "electricity_370",
        "columns": [df_electricity_370.columns.tolist()[4:5]],
        "train_stride": 24,
        "validation_stride": 1,
        "observation_days": 1,
        "future_days": 1,
        "normalize": "minmax",
    },
    # {
    #     'dataset': 'london_smartmeter',
    #     'columns': [df_london_smartmeter.columns.tolist()[:1]],
    #     'train_stride': 24,
    #     'observation_days': 1,
    #     'future_days': 1,
    #     'normalize': 'minmax',
    # },
    # {
    #     'dataset': 'kddcup',
    #     'columns': [df_kddcup.columns.tolist()[:1]],
    #     'train_stride': 24,
    #     'observation_days': 5,
    #     'future_days': 2,
    #     'normalize': 'minmax',
    # },
]

model_config = [
    {
        "_model": FCN_Predictor,
        "hidden_size": 64,
        "_attack_step_multiplier": 1,
    },
    # {
    #     '_model': CNN_Predictor,
    #     'hidden_size': 64,
    #     '_attack_step_multiplier': 1,
    # },
    # {
    #     '_model': TCN_Predictor,
    #     'hidden_size': 64,
    #     'num_levels': 0,
    #     'kernel_size': 6,
    #     'dilation_factor': 2,
    #     'activation': 'relu',
    #     'use_weight_norm': True,
    #     'init_weights': True,
    #     '_attack_step_multiplier': 1,
    #     'dropout': 0.1,
    # },
    # {
    #     '_model': GRU_Predictor,
    #     'hidden_size': 64,
    #     '_attack_step_multiplier': 1,
    # },
    # {
    #     '_model': JitSeq2Seq_Predictor,
    #     'hidden_size': 64,
    #     '_attack_step_multiplier': 1,
    # }
]

start_multi_process(global_config, attack_config, dataset_config, model_config, defense_config, global_config["pool_size"])

Starting 12 processes
def_c {'sign': True, 'defense_name': 'sign'}
Starting attack 0 with config: {'logger_service': 'wandb', 'experiment_name': 'baseline_with_defenses_5-3-2025a', 'seed': 10, 'number_of_batches': 5000, 'warmup_number_of_batches': 0, 'attack_number_of_batches': 1, 'batch_size': 1, 'device': 1, 'verbose': False, 'pool_size': 1, 'run_number': 0, 'total_variation_alpha': 0, 'after_effect': 'none', 'update_model': True, 'load_lti_model': False, 'evaluate_trained_model': True, 'attack_method': 'LTI', 'num_learn_epochs': 250, 'learn_learning_rate': 0.001, 'inversion_batch_size': 1, 'attack_batch_size': 128, 'attack_hidden_size': 3000, 'attack_loss': 'mse', 'learn_optimizer': 'adam', 'learn_lr_decay': '75%', 'inversion_model': 'GradToInputNN', 'attack_targets': True, 'dataset': 'electricity_370', 'columns': ['MT_005'], 'train_stride': 24, 'validation_stride': 1, 'observation_days': 1, 'future_days': 1, 'normalize': 'minmax', 'hidden_size': 64, '_attack_step_multiplier': 1, 'f

def_c {'sign': True, 'defense_name': 'sign'}
Starting attack 1 with config: {'logger_service': 'wandb', 'experiment_name': 'baseline_with_defenses_5-3-2025a', 'seed': 43, 'number_of_batches': 5000, 'warmup_number_of_batches': 0, 'attack_number_of_batches': 1, 'batch_size': 1, 'device': 1, 'verbose': False, 'pool_size': 1, 'run_number': 1, 'total_variation_alpha': 0, 'after_effect': 'none', 'update_model': True, 'load_lti_model': False, 'evaluate_trained_model': True, 'attack_method': 'LTI', 'num_learn_epochs': 250, 'learn_learning_rate': 0.001, 'inversion_batch_size': 1, 'attack_batch_size': 128, 'attack_hidden_size': 3000, 'attack_loss': 'mse', 'learn_optimizer': 'adam', 'learn_lr_decay': '75%', 'inversion_model': 'GradToInputNN', 'attack_targets': True, 'dataset': 'electricity_370', 'columns': ['MT_005'], 'train_stride': 24, 'validation_stride': 1, 'observation_days': 1, 'future_days': 1, 'normalize': 'minmax', 'hidden_size': 64, '_attack_step_multiplier': 1, 'features': [0], 'input_

def_c {'sign': True, 'defense_name': 'sign'}
Starting attack 2 with config: {'logger_service': 'wandb', 'experiment_name': 'baseline_with_defenses_5-3-2025a', 'seed': 28, 'number_of_batches': 5000, 'warmup_number_of_batches': 0, 'attack_number_of_batches': 1, 'batch_size': 1, 'device': 1, 'verbose': False, 'pool_size': 1, 'run_number': 2, 'total_variation_alpha': 0, 'after_effect': 'none', 'update_model': True, 'load_lti_model': False, 'evaluate_trained_model': True, 'attack_method': 'LTI', 'num_learn_epochs': 250, 'learn_learning_rate': 0.001, 'inversion_batch_size': 1, 'attack_batch_size': 128, 'attack_hidden_size': 3000, 'attack_loss': 'mse', 'learn_optimizer': 'adam', 'learn_lr_decay': '75%', 'inversion_model': 'GradToInputNN', 'attack_targets': True, 'dataset': 'electricity_370', 'columns': ['MT_005'], 'train_stride': 24, 'validation_stride': 1, 'observation_days': 1, 'future_days': 1, 'normalize': 'minmax', 'hidden_size': 64, '_attack_step_multiplier': 1, 'features': [0], 'input_

def_c {'prune_rate': 0.1, 'defense_name': 'prune'}
Starting attack 3 with config: {'logger_service': 'wandb', 'experiment_name': 'baseline_with_defenses_5-3-2025a', 'seed': 10, 'number_of_batches': 5000, 'warmup_number_of_batches': 0, 'attack_number_of_batches': 1, 'batch_size': 1, 'device': 1, 'verbose': False, 'pool_size': 1, 'run_number': 3, 'total_variation_alpha': 0, 'after_effect': 'none', 'update_model': True, 'load_lti_model': False, 'evaluate_trained_model': True, 'attack_method': 'LTI', 'num_learn_epochs': 250, 'learn_learning_rate': 0.001, 'inversion_batch_size': 1, 'attack_batch_size': 128, 'attack_hidden_size': 3000, 'attack_loss': 'mse', 'learn_optimizer': 'adam', 'learn_lr_decay': '75%', 'inversion_model': 'GradToInputNN', 'attack_targets': True, 'dataset': 'electricity_370', 'columns': ['MT_005'], 'train_stride': 24, 'validation_stride': 1, 'observation_days': 1, 'future_days': 1, 'normalize': 'minmax', 'hidden_size': 64, '_attack_step_multiplier': 1, 'features': [0], '

def_c {'prune_rate': 0.1, 'defense_name': 'prune'}
Starting attack 4 with config: {'logger_service': 'wandb', 'experiment_name': 'baseline_with_defenses_5-3-2025a', 'seed': 43, 'number_of_batches': 5000, 'warmup_number_of_batches': 0, 'attack_number_of_batches': 1, 'batch_size': 1, 'device': 1, 'verbose': False, 'pool_size': 1, 'run_number': 4, 'total_variation_alpha': 0, 'after_effect': 'none', 'update_model': True, 'load_lti_model': False, 'evaluate_trained_model': True, 'attack_method': 'LTI', 'num_learn_epochs': 250, 'learn_learning_rate': 0.001, 'inversion_batch_size': 1, 'attack_batch_size': 128, 'attack_hidden_size': 3000, 'attack_loss': 'mse', 'learn_optimizer': 'adam', 'learn_lr_decay': '75%', 'inversion_model': 'GradToInputNN', 'attack_targets': True, 'dataset': 'electricity_370', 'columns': ['MT_005'], 'train_stride': 24, 'validation_stride': 1, 'observation_days': 1, 'future_days': 1, 'normalize': 'minmax', 'hidden_size': 64, '_attack_step_multiplier': 1, 'features': [0], '

...

def_c {'dp_epsilon': 0.1, 'defense_name': 'gauss'}
Starting attack 7 with config: {'logger_service': 'wandb', 'experiment_name': 'baseline_with_defenses_5-3-2025a', 'seed': 43, 'number_of_batches': 5000, 'warmup_number_of_batches': 0, 'attack_number_of_batches': 1, 'batch_size': 1, 'device': 1, 'verbose': False, 'pool_size': 1, 'run_number': 7, 'total_variation_alpha': 0, 'after_effect': 'none', 'update_model': True, 'load_lti_model': False, 'evaluate_trained_model': True, 'attack_method': 'LTI', 'num_learn_epochs': 250, 'learn_learning_rate': 0.001, 'inversion_batch_size': 1, 'attack_batch_size': 128, 'attack_hidden_size': 3000, 'attack_loss': 'mse', 'learn_optimizer': 'adam', 'learn_lr_decay': '75%', 'inversion_model': 'GradToInputNN', 'attack_targets': True, 'dataset': 'electricity_370', 'columns': ['MT_005'], 'train_stride': 24, 'validation_stride': 1, 'observation_days': 1, 'future_days': 1, 'normalize': 'minmax', 'hidden_size': 64, '_attack_step_multiplier': 1, 'features': [0], '

def_c {'dp_epsilon': 0.1, 'defense_name': 'gauss'}
Starting attack 8 with config: {'logger_service': 'wandb', 'experiment_name': 'baseline_with_defenses_5-3-2025a', 'seed': 28, 'number_of_batches': 5000, 'warmup_number_of_batches': 0, 'attack_number_of_batches': 1, 'batch_size': 1, 'device': 1, 'verbose': False, 'pool_size': 1, 'run_number': 8, 'total_variation_alpha': 0, 'after_effect': 'none', 'update_model': True, 'load_lti_model': False, 'evaluate_trained_model': True, 'attack_method': 'LTI', 'num_learn_epochs': 250, 'learn_learning_rate': 0.001, 'inversion_batch_size': 1, 'attack_batch_size': 128, 'attack_hidden_size': 3000, 'attack_loss': 'mse', 'learn_optimizer': 'adam', 'learn_lr_decay': '75%', 'inversion_model': 'GradToInputNN', 'attack_targets': True, 'dataset': 'electricity_370', 'columns': ['MT_005'], 'train_stride': 24, 'validation_stride': 1, 'observation_days': 1, 'future_days': 1, 'normalize': 'minmax', 'hidden_size': 64, '_attack_step_multiplier': 1, 'features': [0], '

def_c {'defense_name': 'none'}
Starting attack 9 with config: {'logger_service': 'wandb', 'experiment_name': 'baseline_with_defenses_5-3-2025a', 'seed': 10, 'number_of_batches': 5000, 'warmup_number_of_batches': 0, 'attack_number_of_batches': 1, 'batch_size': 1, 'device': 1, 'verbose': False, 'pool_size': 1, 'run_number': 9, 'total_variation_alpha': 0, 'after_effect': 'none', 'update_model': True, 'load_lti_model': False, 'evaluate_trained_model': True, 'attack_method': 'LTI', 'num_learn_epochs': 250, 'learn_learning_rate': 0.001, 'inversion_batch_size': 1, 'attack_batch_size': 128, 'attack_hidden_size': 3000, 'attack_loss': 'mse', 'learn_optimizer': 'adam', 'learn_lr_decay': '75%', 'inversion_model': 'GradToInputNN', 'attack_targets': True, 'dataset': 'electricity_370', 'columns': ['MT_005'], 'train_stride': 24, 'validation_stride': 1, 'observation_days': 1, 'future_days': 1, 'normalize': 'minmax', 'hidden_size': 64, '_attack_step_multiplier': 1, 'features': [0], 'input_size': 96, 'ou

def_c {'defense_name': 'none'}
Starting attack 10 with config: {'logger_service': 'wandb', 'experiment_name': 'baseline_with_defenses_5-3-2025a', 'seed': 43, 'number_of_batches': 5000, 'warmup_number_of_batches': 0, 'attack_number_of_batches': 1, 'batch_size': 1, 'device': 1, 'verbose': False, 'pool_size': 1, 'run_number': 10, 'total_variation_alpha': 0, 'after_effect': 'none', 'update_model': True, 'load_lti_model': False, 'evaluate_trained_model': True, 'attack_method': 'LTI', 'num_learn_epochs': 250, 'learn_learning_rate': 0.001, 'inversion_batch_size': 1, 'attack_batch_size': 128, 'attack_hidden_size': 3000, 'attack_loss': 'mse', 'learn_optimizer': 'adam', 'learn_lr_decay': '75%', 'inversion_model': 'GradToInputNN', 'attack_targets': True, 'dataset': 'electricity_370', 'columns': ['MT_005'], 'train_stride': 24, 'validation_stride': 1, 'observation_days': 1, 'future_days': 1, 'normalize': 'minmax', 'hidden_size': 64, '_attack_step_multiplier': 1, 'features': [0], 'input_size': 96, '

def_c {'defense_name': 'none'}
Starting attack 11 with config: {'logger_service': 'wandb', 'experiment_name': 'baseline_with_defenses_5-3-2025a', 'seed': 28, 'number_of_batches': 5000, 'warmup_number_of_batches': 0, 'attack_number_of_batches': 1, 'batch_size': 1, 'device': 1, 'verbose': False, 'pool_size': 1, 'run_number': 11, 'total_variation_alpha': 0, 'after_effect': 'none', 'update_model': True, 'load_lti_model': False, 'evaluate_trained_model': True, 'attack_method': 'LTI', 'num_learn_epochs': 250, 'learn_learning_rate': 0.001, 'inversion_batch_size': 1, 'attack_batch_size': 128, 'attack_hidden_size': 3000, 'attack_loss': 'mse', 'learn_optimizer': 'adam', 'learn_lr_decay': '75%', 'inversion_model': 'GradToInputNN', 'attack_targets': True, 'dataset': 'electricity_370', 'columns': ['MT_005'], 'train_stride': 24, 'validation_stride': 1, 'observation_days': 1, 'future_days': 1, 'normalize': 'minmax', 'hidden_size': 64, '_attack_step_multiplier': 1, 'features': [0], 'input_size': 96, '